In [ ]:
import glob
import os
import cv2
import time
import face_detection
import torch
import datetime

In [ ]:
torch.cuda.is_available()

In [ ]:
def draw_faces(im, bboxes):
    for bbox in bboxes:
        x0, y0, x1, y1 = [int(_) for _ in bbox]
        cv2.rectangle(im, (x0, y0), (x1, y1), (0, 0, 255), 2)

In [ ]:
detector = face_detection.build_detector("DSFDDetector", max_resolution=1080)
def detect_and_draw_faces(im):
    dets = detector.detect(im[:, :, ::-1])[:, :4]
    draw_faces(im, dets)

In [ ]:
cap = cv2.VideoCapture("video_in.mp4")

if (not cap.isOpened()):
    print("Error opening video stream or file")

width  = int(cap.get(3))
height = int(cap.get(4))
fps = float(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter("video_out.mp4", fourcc, fps, (width, height))

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
processed_frames = 0
started_at = time.time()

frame_counter = 0
while(cap.isOpened()):
    t0 = time.time()
    ret, frame = cap.read()
    if ret == True:
        frame_counter += 1
        detect_and_draw_faces(frame)
        cv2.imshow('Frame', frame)
        out.write(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame_time = time.time() - t0
        processed_frames += 1
        remaining_frames = total_frames - processed_frames
        print(f"Frames:{processed_frames}/{total_frames} %{100 * processed_frames / total_frames:.2f} FPS:{1/frame_time:.2f} ETA:{datetime.timedelta(seconds=int(remaining_frames * frame_time))}", end="\r")
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()
#print(f"\nDone in {datetime.timedelta(seconds=int(time.time() - started_at))}{space_buffer}")